#YG3 API Guide

##The YG3 API currently has 3 AI models available:

###Elysia (Text):

Pure text model using chat-style messages. Ideal for conversation, reasoning, and idea generation.

###Taurus (Vision):

Vision-enabled chat model that accepts images + text inside the same message block.

###Vani (Image Generation):

Creative image generator using a dedicated /images/generations endpoint, returning base64 images.

*Learn how to get started with these powerful models by reading below!*

## 📋 Table of Contents
1. Setup & Authentication
2. Elysia
3. Taurus
4. Vani
5. Advanced
6. Tips

## 1️⃣ Setup & Authentication

In [1]:
!pip install requests pillow -q

import requests, base64, json, os
from PIL import Image
from io import BytesIO
from IPython.display import display

API_KEY = "sk-bNO67hFhS4R-MvMjEEvA41RsxA5td7nbB8l3dmUq2Us"
API_BASE = "https://elysia-api.ngrok.io/api/public/v1"

headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}

print("Setup complete.")

Setup complete.


## 2️⃣ Elysia - Conversational AI

In [2]:
def chat_with_elysia(message, temperature=0.7, max_tokens=500):
    payload = {
        "model": "elysia",
        "messages": [{"role": "user", "content": message}],
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    r = requests.post(f"{API_BASE}/chat/completions", headers=headers, json=payload)
    print(r.json())

## 3️⃣ Taurus - Vision Understanding

In [3]:
def analyze_image_with_taurus(image_url, question):
    img = Image.open(BytesIO(requests.get(image_url).content))
    buffer = BytesIO()
    img.save(buffer, format="PNG")
    img_b64 = base64.b64encode(buffer.getvalue()).decode()
    payload = {
        "model": "taurus",
        "messages": [{
            "role": "user",
            "content": [
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{img_b64}"}},
                {"type": "text", "text": question}
            ]
        }]
    }
    r = requests.post(f"{API_BASE}/chat/completions", headers=headers, json=payload)
    display(img)
    print(r.json())

## 4️⃣ Vani - Image Generation

In [4]:
def create_image_with_vani(prompt, size="1024x1024"):
    payload = {"model": "vani", "prompt": prompt, "size": size, "response_format": "b64_json"}
    r = requests.post(f"{API_BASE}/images/generations", headers=headers, json=payload)
    img_bytes = base64.b64decode(r.json()['data'][0]['b64_json'])
    img = Image.open(BytesIO(img_bytes))
    display(img)

## 5️⃣ Advanced Examples

In [ ]:
# Add advanced workflows here as needed.

## 6️⃣ Tips & Best Practices
- Ask open-ended questions for Elysia
- Use clear images for Taurus
- Use descriptive prompts for Vani

#**Elysia** is a pure text-generation model, similar in interface to OpenAI’s Chat Completions API.

It expects a list of messages in the messages=[...] format and returns structured chat output.

How calling Elysia works:

You send { "model": "elysia", "messages": [...] }

Messages must follow the chat completion structure:
{"role": "user" | "assistant" | "system", "content": "..."}

Responses include tokens + structured choices just like OpenAI

It returns text only — no images, no multimedia objects

When to use Elysia:

Conversation

Ideation

Reflection

Reasoning

Generating prompts for Vani

Follow-ups after Taurus image descriptions

💡 Think of Elysia as the conversational reasoning engine.


#**Taurus** extends the chat completion structure to support images.
It uses the same endpoint as Elysia (/chat/completions) but expects an additional media object inside content.

How calling Taurus works:

You send a message with both an image and text

The image must be base64 encoded or a URL

The content list of the user role contains different “parts,” e.g.:

{
  "role": "user",
  "content": [
    { "type": "image_url", "image_url": { "url": "data:image/png;base64,..." }},
    { "type": "text", "text": "What is in this image?" }
  ]
}


Key differences from Elysia:

Uses the same API route, but content is multi-modal

Must pass at least one image in the message

Returns text, not images

When to use Taurus:

Understanding content in images

Extracting context / objects / scenes

OCR-like tasks

Feeding visual descriptions into Elysia for deeper reasoning

💡 Think of Taurus as the "eyes" of the system.

#**Vani** uses a different endpoint from the other two models because it does not participate in chat-style turns.

Instead, you call:

POST /images/generations


This endpoint accepts a prompt and generates an image in base64.

How calling Vani works:

You send:

{
  "model": "vani",
  "prompt": "A beautiful mountain lake at sunrise",
  "size": "1024x1024",
  "response_format": "b64_json"
}


It returns an image, encoded as base64

You decode + display the image using PIL/Jupyter

Key differences from Elysia and Taurus:

Not chat-based

Only accepts text prompt (no message list)

Returns image outputs, not text

Uses a generation endpoint instead of a chat endpoint

When to use Vani:

Artwork

Concept visualization

Rendering ideas generated by Elysia

Creating image series or storyboards

💡 Think of Vani as the artist of the system.